## Login to Hugging Face

In [1]:
from dotenv import load_dotenv
import os
from huggingface_hub import login

load_dotenv()
token = os.getenv("HUGGINGFACE_TOKEN")
login(
    token=token, # ADD YOUR TOKEN HERE
    add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /home/pathfinder/.cache/huggingface/token
Login successful


## Imports

In [2]:
from IPython.display import display, Markdown

# pytorch
import torch

# huggingface
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)

## Device

In [3]:
# Device setup
device = (
    "cuda:0" if torch.cuda.is_available() else # Nvidia GPU
    "mps" if torch.backends.mps.is_available() else # Apple Silicon GPU
    "cpu"
)
print(f"Device = {device}")

Device = cuda:0


In [4]:
# Flash Attention Implementation
if device == "cuda:0":
    if torch.cuda.get_device_capability()[0] >= 8: # Ampere, Ada, or Hopper GPUs
        attn_implementation = "flash_attention_2"
        torch_dtype = torch.bfloat16
    else:
        attn_implementation = "eager"
        torch_dtype = torch.float16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float32
print(f"Attention Implementation = {attn_implementation}")

Attention Implementation = flash_attention_2


## Hyperparameters

In [5]:
################################################################################
# Tokenizer parameters
################################################################################
max_length=8192
padding="do_not_pad" # "max_length", "longest", "do_not_pad"
truncation=True

################################################################################
# Generation parameters
################################################################################
num_return_sequences=1
max_new_tokens=1024
do_sample=True # True for sampling, False for greedy decoding
temperature=0.6
top_k=0 # not recommended
top_p=0.9
repetition_penalty=1.1

################################################################################
# bitsandbytes parameters
################################################################################
load_in_4bit=True
bnb_4bit_compute_dtype=torch_dtype
bnb_4bit_quant_type="nf4" # "nf4", #fp4"
bnb_4bit_use_double_quant=True

## Model

In [6]:
# Model List

# gemma variants
# "google/gemma-1.1-7b-it"
# "google/codegemma-7b-it"

# llama variants
# "meta-llama/Meta-Llama-3-8B" // downloaded
# "meta-llama/Meta-Llama-3-8B-Instruct" // downloaded
# "codellama/CodeLlama-7b-Instruct-hf"
# "PathFinderKR/Waktaverse-Llama-3-KO-8B-Instruct"

# mistral variants
# "mistralai/Mistral-7B-Instruct-v0.2"

# openELM variants
# "apple/OpenELM-3B-Instruct" // downloaded

# solar variants
# "upstage/SOLAR-10.7B-Instruct-v1.0" // downloaded
# "PathFinderKR/Waktaverse-SOLAR-KO-10.7B-Instruct"

In [7]:
model_id = "PathFinderKR/Waktaverse-Llama-3-KO-8B-Instruct"

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.padding_side = "right"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=load_in_4bit,
    bnb_4bit_compute_dtype=bnb_4bit_compute_dtype,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_use_double_quant=bnb_4bit_use_double_quant
)

In [10]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map=device,
    attn_implementation=attn_implementation,
    torch_dtype=torch_dtype,
    quantization_config=quantization_config
)

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [11]:
# Display the model architecture
display(Markdown(f'```{model}```'))

```LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaFlashAttention2(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Linear(in_features=4096, out_features=128256, bias=False)
)```

## Inference

In [12]:
def generate_response(system ,user):
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": user}
    ]
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )
    
    input_ids = tokenizer.encode(
        prompt,
        max_length=max_length,
        padding=padding,
        truncation=truncation,
        add_special_tokens=False,
        return_tensors="pt"
    ).to(device)
    
    outputs = model.generate(
        input_ids=input_ids,
        pad_token_id=tokenizer.eos_token_id,
        num_return_sequences=num_return_sequences,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        repetition_penalty=repetition_penalty
    )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=False)

In [13]:
system_prompt = "다음 지시사항에 대한 응답을 작성해주세요."

In [14]:
user_prompt = "피보나치 수열에 대해 설명해주세요."

In [15]:
response = generate_response(system_prompt, user_prompt)
print(response)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

다음 지시사항에 대한 응답을 작성해주세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

피보나치 수열에 대해 설명해주세요.<|eot_id|><|start_header_id|>assistant<|end_header_id|>"

피보나치 수열은 0과 1로 시작하며, 각 항이 이전 두 개의 항을 더한 값으로 정의되는 수열입니다. 피보나치 수열은 수학에서 가장 유명한 수열 중 하나이며, 특히 재귀 함수와 관련된 문제에 자주 사용됩니다.

피보나치 수열의 첫 번째 12개의 항은 다음과 같습니다:

0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89

피보나치 수열은 0과 1로 시작하여, 각 항이 이전 두 개의 항을 더한 값으로 정의됩니다. 예를 들어, 7번째 항은 (5 + 2) = 7이고, 8번째 항은 (8 + 3) = 11입니다.

피보나치 수열은 수학적 구조를 가지고 있으며, 이 구조는 자연스럽게 나타납니다. 피보나치 수열은 수학에서 매우 중요하며, 다양한 분야에서 적용될 수 있습니다. 예를 들어, 피보나치 수열은 재귀 함수와 관련된 문제에 자주 사용되며, 컴퓨터 프로그래밍에서도 자주 활용됩니다. 또한, 피보나치 수열은 수학의 다른 영역에서도 중요한 역할을 하며, 이를 통해 새로운 수학적인 발견들을 이루어낼 수 있습니다.

피보나치 수열은 많은 문화에서 영향을 미쳤으며, 음악, 문학, 아트 등 다양한 분야에서 표현되어져 있습니다. 피보나치 수열은 수학의 기본 원리와 연결되어 있으며, 이를 통해 수학의 주요 개념과 관계를 이해하는 데 도움이 됩니다.

결론적으로, 피보나치 수열은 수학에서 가장 유명한 수열 중 하나이며, 재귀 함수와 관련된 문제에 자주 사용됩니다. 피보나치 수열은 수학적 구조를 가지고 있으며, 이 구조는 자연스럽게 나타나고, 다양한 분야에서 중요한 역할을 합니다. 피보나치 수열은

## Multi-turn Conversation

In [16]:
system_prompt = "다음 지시사항에 대한 응답을 작성해주세요."
conversation_history = [{"role": "system", "content": system_prompt}]

In [17]:
def generate_conversation(user):
    global conversation_history
    conversation_history.append({"role": "user", "content": user})
    prompt = tokenizer.apply_chat_template(
        conversation_history,
        tokenize=False,
        add_generation_prompt=False
    )
    
    input_ids = tokenizer.encode(
        prompt,
        max_length=max_length,
        padding=padding,
        truncation=truncation,
        add_special_tokens=False,
        return_tensors="pt"
    ).to(device)
    
    outputs = model.generate(
        input_ids=input_ids,
        pad_token_id=tokenizer.eos_token_id,
        num_return_sequences=num_return_sequences,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        repetition_penalty=repetition_penalty
    )
    
    assistant_response = tokenizer.decode(outputs[0], skip_special_tokens=False).split("<|start_header_id|>assistant<|end_header_id|>")[-1]
    conversation_history.append({"role": "assistant", "content": assistant_response})
    return assistant_response

In [18]:
user_input = "이번 주말에 할 수 있는 취미 활동을 추천해주세요."
response = generate_conversation(user_input)
print(response)



여러분의 취미를 고려하여 다양한 주말 활동을 추천합니다:

1. 산책: 도시에서 멀리 떨어진 곳으로 이동하는 것이 좋습니다. 자연을 감상하고 바람을 느끼며 휴식을 취할 수 있습니다.

2. 요리: 새롭게 만드는 요리를 시도하세요. 인터넷에서 레시피를 찾거나 가족과 함께 만들 수 있습니다. 맛있는 음식은 즐거운 시간을 보낼 수 있어요!

3. 운동: 주말에는 스트레스를 해소하기 위해 운동을 하세요. 자전거, 피트니스 센터 또는 친구와 함께 야외에서 운동을 시작하세요.

4. 여행: 가까운 장소나 멀리 가서 새로운 장소를 방문하세요. 새로운 경험을 얻고 새로운 사람들을 만나기 위한 좋은 기회가 될 수 있습니다.

5. 미술: 도안, 그림, 사진 등을 그려보세요. 창의적인 능력을 발휘하면서 재미를 누릴 수 있습니다.

6. 게임: 친구들과 온라인 게임이나 보드 게임을 하세요. 가족과 함께하면 더 재미있을 수 있습니다.

7. 음악: 악기를 배우거나 노래를 부르세요. 음악은 스트레스를 해소하고 즐거움을 주는 것으로 알려져 있습니다.

8. 식물 키우기: 집에서 식물을 키워보세요. 이로 인해 집 안의 공기 청정 효과와 함께 스트레스를 줄일 수 있습니다.

9. 독서: 책을 읽어보세요. 새로운 지식과 이야기를 얻으면 좋습니다.

10. 영화보기: 최근 개봉한 영화를 볼 수도 있고, 오래된 클래식 영화도 재미있을 수 있습니다.

11. 일상 생활에서 불필요한 물품을 정리하세요. 이것은 공간을 정리하고 정신적으로도 긍정적인 영향을 미칠 수 있습니다.

12. 여행 계획을 세우세요. 가까운 장소나 멀리 가서 새로운 장소를 방문하려고 계획하세요. 새로운 경험을 얻고 새로운 사람들을 만나기 위한 좋은 기회가 될 수 있습니다.

13. 커피 테라스에서 시간을 보내세요. 따뜻한 커피와 좋은 분위기로 휴식을 취하세요.

14. 요리 강습: 요리에 관심이 있다면 요리 강습 프로그램을 따라해보세요. 새로운 요리를 배울 수 있으며, 새로운 요리를 만들어 가족과 함께 먹을 수 있습니다.


In [19]:
user_input = "추천한 취미 활동 중 건강에 도움이 되는 것은 무엇인가요?"
response = generate_conversation(user_input)
print(response)



Recommendations for healthy hobbies include:

1. Exercise: Cycling, fitness center, outdoor sports, etc.
2. Camping: Outdoor activities can be beneficial for physical and mental health.
3. Yoga or meditation: Mindfulness exercises that improve overall well-being.
4. Reading books: Intellectual stimulation can reduce stress and improve mental health.
5. Traveling: Explore new places and cultures to broaden your horizons and gain new experiences.
6. Gardening: Planting flowers and plants can be therapeutic and provide a sense of accomplishment.
7. Cooking: Healthy eating habits can be developed through cooking at home with fresh ingredients.
8. Photography: Capturing moments and beauty in nature can be a great way to relax and appreciate the world around you.

These activities are not only fun but also have benefits for our physical and mental health. Remember to choose activities that bring joy and fulfillment while respecting your body's limitations.<|eot_id|>
